In [ ]:
from filemanager import get_file_paths as gfp
import tensorflow as tf

import io
import os
import numpy as np
from PIL import Image


In [ ]:
def write_tfrecords(x, y, filename):
    writer = tf.io.TFRecordWriter(filename)

    for image, label in zip(x, y):
        example = tf.train.Example(features=tf.train.Features(
            feature={
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tobytes()])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
            }))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
dataset = gfp.File_Control.searchAllFilesInDirectoryByDir("/raid/food_competition_KT_set1/train", "jpg")
label_dict = {}
for i in range(len(dataset)):
    label = os.path.dirname(dataset[i][0])
    label = os.path.basename(label)
    label_dict[str(i)] = label

In [ ]:
print(label_dict)

In [ ]:
def elementCount(list):
    count = 0
    for element in list:
        count += len(element)
    return count

print("총 데이터 개수 :", elementCount(dataset))
y = np.ndarray((elementCount(dataset)), dtype=np.int32)
x = np.ndarray((elementCount(dataset), 256, 256, 3), dtype=np.float32)

dataset_len = elementCount(dataset)
index = 0
RESIZE_WIDTH = 256
RESIZE_HEIGHT = 256
CHANNEL = 3
for i in range(len(dataset)):
    for data in dataset[i]:
        img = Image.open(data)
        img = img.convert('RGB')
        img = img.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
        #img = preprocess_input(np.array(img))
        y[index] = i
        x[index] = img
        index = index + 1
        gfp.Progress_Bar.printProgressBar(index, dataset_len)

y_unique_num = np.unique(y, axis=0)
y_unique_num = y_unique_num.shape[0]

y_encoded = np.eye(y_unique_num)[y] # One-Hot Encoding
print(x.shape, y.shape)

In [ ]:
print("tfrecord saving...")
write_tfrecords(x, y, '/raid/food_competition_KT_set1/kt_food.tfrecord')
print("Complete!")